In [1]:
from models import *
from DScontrol import *
import matplotlib.pyplot as plt
import warnings

In [ ]:
models = [
    EdgeBlur(15, 5, 15, 1, 5),
    FourierGabor(5, 0, 5)
]

predict = {
    'EdgeBlur':[],
    'FourierGabor':[]
}
total = len(DS)
for i, filename in enumerate(DS.filename):
    image = get(filename)
    for modelName, model in zip(predict.keys(), models):
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            predict[modelName].append(model.predict(image))

    print(f'{i+1}/{total}|{(i/total*100):.2f}%', end='\r')